## 0. Libarary 불러오기 및 경로설정

In [1]:
import multiprocessing
import os
from importlib import import_module

import pandas as pd
import torch
from torch.utils.data import DataLoader
import numpy as np

from dataset import TestDataset, MaskBaseDataset
from torchmetrics.classification import MulticlassF1Score

In [2]:
model_paths = ['/opt/ml/workspace/teajun/level1_imageclassification_cv-level-cv-19/model/1101ConvNext_Small_resize/3_best_f1_0.833_0.897.pth', '/opt/ml/workspace/teajun/level1_imageclassification_cv-level-cv-19/model/1101ConvNext_Small_resize/3_best_f1_0.833_0.897.pth', '/opt/ml/workspace/teajun/level1_imageclassification_cv-level-cv-19/model/1101ConvNext_Small_resize/3_best_f1_0.833_0.897.pth']
model_structs = ['ConvNext_Small', 'ConvNext_Small', 'ConvNext_Small']

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
num_classes = MaskBaseDataset.num_classes  # 18
data_dir = '/opt/ml/input/data/eval'
img_root = os.path.join(data_dir, 'images')
info_path = os.path.join(data_dir, 'info.csv')
info = pd.read_csv(info_path)
img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]
batch_size = 64
resize=(256, 192)
dataset = TestDataset(img_paths, resize)

loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False,
)


img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [4]:
prediction_array_3d = []

softmax = torch.nn.Softmax(dim=1)
for model_struct, model_path in zip(model_structs, model_paths):
    model_cls = getattr(import_module('model'), model_struct)
    model = model_cls(num_classes)
    # 모델 생성 및 파라미터 주입
    model.load_state_dict(torch.load(model_path), model_struct)
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        temp_prediction = []
        for idx, images in enumerate(loader):
            images = images.to(device)
            pred = softmax(model(images))
            temp_prediction.extend(pred.cpu().numpy())
        prediction_array_3d.append(temp_prediction)
prediction_array_3d = np.array(prediction_array_3d)
prediction = prediction_array_3d.sum(axis=0)
prediction = prediction.argmax(axis=1)
info['ans'] = prediction
#             batch_idx = batch_size * idx
            # images.shape[0] 는 64. 즉, 배치 만큼만 채워 넣는다.
#             prediction_array[batch_idx: batch_idx + images.shape[0], :] = preds.astype(int)



In [5]:
info.head()
print('hihi')

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12


In [6]:
m = torch.nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)

In [8]:
pred.shape

torch.Size([56, 18])